## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
from math import ceil
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = 'drive/MyDrive/Data_analitic/Python/statistical_hypothesis/cookie_cats.csv'
dft = pd.read_csv(data_path)

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [18]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [4]:
effect_size = sms.proportion_effectsize(0.20, 0.19)

In [7]:
effect_size

np.float64(0.025241594409087353)

In [8]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(f'Кількість користувачі, яка необхідна для тестування {required_n}')

Кількість користувачі, яка необхідна для тестування 24638


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [14]:
avg_retention = dft.groupby('version')['retention_7'].mean()
print(avg_retention)

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64


##Гіпотеза
- версія гри gate_30 дає 19% утримання
- версія гри gate_40 дає 18.2% утримання
- формування гіпотез:
   - Но - 𝑝=𝑝0 (перенсення воріт на 10 рівнів не повпливало на залученість гравців)
   - Н1 - 𝑝≠𝑝0 (перенесення воріт на 10 рівнів змінило утримання гравців у грі)

3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [23]:
control_results = dft[dft['version'] == 'gate_30']['retention_7']
treatment_results = dft[dft['version'] == 'gate_40']['retention_7']

In [24]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

In [26]:
z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи gate_30: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи gate_40: [{lower_treat:.3f}, {upper_treat:.3f}]')

z statistic: 3.16
p-value: 0.002
Довірчий інтервал 95% для групи gate_30: [0.187, 0.194]
Довірчий інтервал 95% для групи gate_40: [0.178, 0.186]


##Висновок
- p-value = 0.002 < 0.05, то ми відкидаємо нульову гіпотезу, на користь альтернативної гіпротези, що утримання на 7 день змінилося. Є статистична значущість між поведінкою користувачів у різних версіях гри на 7 день.
- ні, довірчі інтервали не перетинаються. Це підтверджує сатистично значущу різницю між групами користувачів.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [6]:
crosstab = pd.crosstab(dft['version'], dft['retention_7'])

In [7]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f'χ² = {chi2:.3f}')
print(f'p-value = {p:.5f}')
print(f'Ступені свободи = {dof}')
print(f'Очікувані частоти:\n', expected)

χ² = 9.959
p-value = 0.00160
Ступені свободи = 1
Очікувані частоти:
 [[36382.90257127  8317.09742873]
 [37025.09742873  8463.90257127]]


##Гіпотеза
- Но - версія гри, а точніше переміщення воріт на 10 рівнів пізніше не вплинуло на залученість учасників
- Н1 - версія гри, вплиуло на залученість користувачів на 7 день

##Висновок
- p-value = 0.0016 < 0.05, то ми відкидаємо нульову гіпотезу, на користь альтернативної гіпротези, що утримання на 7 день змінилося. Є статистична значущість між поведінкою користувачів у різних версіях гри на 7 день.

In [8]:
crosstab_1 = pd.crosstab(dft['version'], dft['retention_1'])

In [13]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab_1)

print(f'χ² = {chi2:.3f}')
print(f'p-value = {p:.5f}')
print(f'Ступені свободи = {dof}')
print(f'Очікувані частоти:\n' , expected)

χ² = 3.159
p-value = 0.07550
Ступені свободи = 1
Очікувані частоти:
 [[24799.13514952 19900.86485048]
 [25236.86485048 20252.13514952]]


Проте, результати на 1 день відрізняються:
- p-value = 0.07550 > 0.05, отже ми не відкидаємо нульову гіпотезу.
Це означає, що немає статистично значущої залежності між версією гри та тим, чи користувач повернувся в перший день після встановлення гри.